# 麥克連大盤脈動_part_4_三大法人買賣超  

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import duckdb

In [2]:
from finlab import data

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [4]:
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [6]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [7]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
# ipython 載入 sql 擴充, 並設置 duckdb 連接
%load_ext sql

%sql conn_duckdb

In [ ]:
%%sql
-- 方法1：查詢當前數據庫中的所有表
SHOW TABLES;

## 三大法人買賣超  

In [10]:
# 三大法人買賣超 from Finlab
major_institution_info = data.get('institutional_investors_trading_all_market_summary:買賣超', save_to_storage=True)
major_institution_info.reset_index(inplace=True)
major_institution_info = major_institution_info[['date', '上市自營商(自行買賣)', '上市自營商(避險)', '上市投信', '上市外資及陸資(不含外資自營商)']]
# major_institution_info

In [ ]:
# 將date以外的欄位金額換算成億元
major_institution_info['上市自營商(自行買賣)'] = major_institution_info['上市自營商(自行買賣)'] / 100000000
major_institution_info['上市自營商(避險)'] = major_institution_info['上市自營商(避險)'] / 100000000
major_institution_info['上市投信'] = major_institution_info['上市投信'] / 100000000
major_institution_info['上市外資及陸資(不含外資自營商)'] = major_institution_info['上市外資及陸資(不含外資自營商)'] / 100000000
major_institution_info

In [12]:
# 欄位名稱rename
major_institution_info.rename(
    columns={
        'date': 'Date', 
    }
    , inplace=True
)

In [ ]:
major_institution_info

In [ ]:
%%sql
-- 先检查表是否存在，如果不存在则创建
CREATE TABLE IF NOT EXISTS tw_sotck_barometer_part_04 (
    Date DATE PRIMARY KEY,
    上市自營商_自行買賣 DOUBLE,
    上市自營商_避險 DOUBLE,
    上市投信 DOUBLE,
    上市外資及陸資_不含外資自營商 DOUBLE,
    外資買賣超金額_多空分數 INTEGER
);

-- 使用UPSERT方式插入数据
INSERT OR REPLACE INTO tw_sotck_barometer_part_04
SELECT 
    CAST(Date AS DATE) AS Date,
    "上市自營商(自行買賣)" as 上市自營商_自行買賣,
    "上市自營商(避險)" as 上市自營商_避險,
    "上市投信" as 上市投信,
    "上市外資及陸資(不含外資自營商)" as 上市外資及陸資_不含外資自營商,
    CASE 
        WHEN "上市外資及陸資(不含外資自營商)" < -200 THEN -2  -- 外资卖超超过200亿，标示两个空(-2分)
        WHEN "上市外資及陸資(不含外資自營商)" < 0 THEN -1        -- 外资卖超少于200亿，标示一个空(-1分)
        WHEN "上市外資及陸資(不含外資自營商)" > 150 THEN 2     -- 外资买超超过150亿，标示两个多(2分)
        WHEN "上市外資及陸資(不含外資自營商)" > 0 THEN 1          -- 外资买超少于150亿，标示一个多(1分)
        ELSE 0                                              -- 其他情况为0分
    END AS 外資買賣超金額_多空分數
FROM major_institution_info;

In [ ]:
%%sql
SELECT * FROM tw_sotck_barometer_part_04 ORDER BY Date DESC LIMIT 10;

In [16]:
# 關閉資料庫連線
conn_duckdb.close()